### STEP 1: Set UP

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

In [ ]:
%%writefile ~/aou_dsub.bash

#!/bin/bash

# This shell function passes reasonable defaults for several dsub parameters, while
# allowing the caller to override any of them. It creates a nice folder structure within
# the workspace bucket for dsub log files.

# --[ Parameters ]--
# any valid dsub parameter flag

#--[ Returns ]--
# the job id of the job created by dsub

#--[ Details ]--
# The first five parameters below should always be those values when running on AoU RWB.

# Feel free to change the values for --user, --regions, --logging, and --image if you like.

# Note that we insert some job data into the logging path.
# https://github.com/DataBiosphere/dsub/blob/main/docs/logging.md#inserting-job-data

function aou_dsub () {

  # Get a shorter username to leave more characters for the job name.
  local DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

  # For AoU RWB projects network name is "network".
  local AOU_NETWORK=network
  local AOU_SUBNETWORK=subnetwork

  dsub \
      --provider google-cls-v2 \
      --user-project "${GOOGLE_PROJECT}"\
      --project "${GOOGLE_PROJECT}"\
      --image 'marketplace.gcr.io/google/ubuntu1804:latest' \
      --network "${AOU_NETWORK}" \
      --subnetwork "${AOU_SUBNETWORK}" \
      --service-account "$(gcloud config get-value account)" \
      --user "${DSUB_USER_NAME}" \
      --regions us-central1 \
      --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
      "$@"
}

### STEP 2: Copy over files to annotate

In [ ]:
%%bash
source ~/aou_dsub.bash

USER_NAME=${USER_NAME:-$(whoami)}
JOB_NAME="copy-vcf-files-${USER_NAME}"

# Define source and destination bucket paths
ORIGINAL_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/newID_PLINK"
SECURE_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_newID"

aou_dsub \
  --name "${JOB_NAME}_copy" \
  --image "gcr.io/google.com/cloudsdktool/cloud-sdk" \
  --logging "${SECURE_BUCKET}/logging" \
  --min-cores 2 \
  --min-ram 4 \
  --boot-disk-size 50 \
  --disk-size 500 \
  --command "
    set -ex
    echo 'Starting VCF file copy process...'
    
    for CHR in {1..22}; do
      echo \"Copying chromosome \$CHR...\"
      gsutil -u ${GOOGLE_PROJECT} cp ${ORIGINAL_BUCKET}/exome_v8.chr\${CHR}.new_id.split_multiallelic.sites_only.pass_qc.vcf.gz ${SECURE_BUCKET}/
    done
    
    echo 'VCF file copy process completed successfully.'
  " \
  --wait


In [ ]:
%%bash

# Replace BUCKET_PATH with your actual GCS bucket path
BUCKET_PATH="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache"

# Download the required cache files from Ensembl to your local machine or VM with minimal output
wget --progress=dot:giga -P ./ \
  https://ftp.ensembl.org/pub/release-113/variation/indexed_vep_cache/homo_sapiens_vep_113_GRCh38.tar.gz

wget --progress=dot:giga -P ./ \
  https://ftp.ensembl.org/pub/release-113/variation/indexed_vep_cache/homo_sapiens_merged_vep_113_GRCh38.tar.gz

wget --progress=dot:giga -P ./ \
  https://ftp.ensembl.org/pub/release-113/variation/indexed_vep_cache/homo_sapiens_refseq_vep_113_GRCh38.tar.gz

# Upload the files to your GCS bucket
gsutil cp homo_sapiens_*.tar.gz $BUCKET_PATH/

### STEP 3: Run VEP to annotate and NOTE, adjust your parameters based off of how big your files are and how much resources are available 

#### Also this step includes extracting your cache files

In [ ]:
%%bash
set -euo pipefail
source ~/aou_dsub.bash

#This is just for chr1 since it is massive

USER_NAME=${USER_NAME:-$(whoami)}
JOB_NAME="vep-test-lof-${USER_NAME}"

# Define bucket locations for required files
PLUGIN_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/plugins"
CACHE_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache"
VCF_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_newID"

EXT_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache/extracted"
OUTPUT_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/output"

aou_dsub \
  --name "${JOB_NAME}" \
  --project "${GOOGLE_PROJECT}" \
  --image "gcr.io/ritchie-aou-psom-9015/ensembl-vep:latest" \
  --logging "${OUTPUT_BUCKET}/logging" \
  --min-cores 32 \
  --min-ram 128 \
  --disk-size 3000 \
  --input-recursive input_lof_plugin="${PLUGIN_BUCKET}" \
  --input-recursive input_cache="${CACHE_BUCKET}" \
  --input-recursive input_vcf="${VCF_BUCKET}" \
  --output-recursive extracted_cache="${EXT_BUCKET}" \
  --output-recursive output="${OUTPUT_BUCKET}" \
  --command "
    set -ex

    # Localized paths
    PLUGIN_DIR=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/plugins
    # Expect LoFTEE files in the loftee subdirectory
    HUMAN_ANCESTOR_FILE=\${PLUGIN_DIR}/human_ancestor.fa.gz
    HUMAN_ANCESTOR_INDEX=\${PLUGIN_DIR}/human_ancestor.fa.gz.fai
    CONSERVATION_FILE=\${PLUGIN_DIR}/loftee.sql
    GERP_FILE_PATH=\${PLUGIN_DIR}/gerp_conservation_scores.homo_sapiens.GRCh38.bw
    CACHE_DIR=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache
    EXTRACTED_CACHE_DIR=/mnt/data/output/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache/extracted
    # Define test VCF for chromosome 1
    TEST_VCF=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_newID/exome_v8.chr1.new_id.split_multiallelic.sites_only.pass_qc.vcf.gz

    # Export environment variables needed for VEP
    export PERL5LIB=/opt/vep/src/ensembl-vep/modules:/opt/vep/src/bioperl-live:\$PERL5LIB:/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e31ab9cc/vep/plugins
    # Use the loftee subdirectory for LOFTEE files
    export LOFTEE_DIR=\${PLUGIN_DIR}/loftee

    echo 'Listing contents of the plugins directory:'
    ls -lh \${PLUGIN_DIR}

    echo 'Listing contents of the cache directory:'
    ls -lh \${CACHE_DIR}

    echo 'Extracting VEP cache files:'
    CACHE_FILES=(homo_sapiens_vep_113_GRCh38.tar.gz homo_sapiens_merged_vep_113_GRCh38.tar.gz homo_sapiens_refseq_vep_113_GRCh38.tar.gz)
    for FILE in \"\${CACHE_FILES[@]}\"; do
      FULL_PATH=\"\${CACHE_DIR}/\${FILE}\"
      if [[ -f \"\${FULL_PATH}\" ]]; then
        echo \"Extracting \${FULL_PATH}...\"
        tar -xzf \"\${FULL_PATH}\" -C \"\${EXTRACTED_CACHE_DIR}\"
      else
        echo \"WARNING: File \${FULL_PATH} not found! Skipping...\"
      fi
    done

    echo 'Listing extracted cache contents:'
    ls -lh \${EXTRACTED_CACHE_DIR}

    echo 'Listing the test VCF file:'
    ls -lh \${TEST_VCF}

    # Define the main output file
    OUTPUT_FILE=/mnt/data/output/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/output/AOU_v8.finalAnnot.chr1.new_id.split_multiallelic.sites_only.pass_qc.txt

    echo 'Running VEP with the LoF plugin on chromosome 1...'
    perl /opt/vep/src/ensembl-vep/vep \\
      -i \${TEST_VCF} \\
      -o \${OUTPUT_FILE} \\
      --warning_file /tmp/vep_warnings.txt \\
      --plugin LoF,loftee_path:\${PLUGIN_DIR}/loftee,human_ancestor_fa:\${HUMAN_ANCESTOR_FILE},human_ancestor_index:\${HUMAN_ANCESTOR_INDEX},conservation_file:\${CONSERVATION_FILE},gerp_bigwig:\${GERP_FILE_PATH} \\
      --plugin CADD,snv=\${PLUGIN_DIR}/whole_genome_SNVs_inclAnno.tsv.gz,indels=\${PLUGIN_DIR}/whole_genome_SNVs_inclAnno.tsv.gz \\
      --plugin dbNSFP,\${PLUGIN_DIR}/dbNSFP/dbNSFP4.9a_grch38.gz,Ensembl_transcriptid,Uniprot_acc,VEP_canonical,LRT_pred,SIFT_pred,MutationTaster_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_score \\
      --plugin SpliceAI,snv=\${PLUGIN_DIR}/spliceai_scores.raw.snv.hg38.vcf.gz,indel=\${PLUGIN_DIR}/spliceai_scores.raw.indel.hg38.vcf.gz \\
      --everything \\
      --buffer_size 40000 \\
      --fork 24 \\
      --minimal \\
      --offline \\
      --dir_cache \${EXTRACTED_CACHE_DIR} \\
      --no_escape \\
      --fasta \${PLUGIN_DIR}/Homo_sapiens_assembly38.fasta \\
      --hgvs \\
      --cache \\
      --format vcf \\
      --force_overwrite \\
      --tab \\
      --dir_plugins \${PLUGIN_DIR}

    echo 'VEP run completed. Check the output at:' \${OUTPUT_FILE}
  "

In [ ]:
%%bash
set -euo pipefail
source ~/aou_dsub.bash

USER_NAME=${USER_NAME:-$(whoami)}
JOB_PREFIX="vep-test-lof-${USER_NAME}"

# Define bucket locations for required files
PLUGIN_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/plugins"
CACHE_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache"
VCF_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_newID"

EXT_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache/extracted"
OUTPUT_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/output"

# Function to run a VEP job for a given chromosome number.
run_vep_job() {
  local CHR=$1
  local JOB_NAME="${JOB_PREFIX}-chr${CHR}"
  
  aou_dsub \
    --name "${JOB_NAME}" \
    --project "${GOOGLE_PROJECT}" \
    --image "gcr.io/ritchie-aou-psom-9015/ensembl-vep:latest" \
    --logging "${OUTPUT_BUCKET}/logging" \
    --min-cores 32 \
    --min-ram 128 \
    --disk-size 3000 \
    --input-recursive input_lof_plugin="${PLUGIN_BUCKET}" \
    --input-recursive input_cache="${CACHE_BUCKET}" \
    --input-recursive input_vcf="${VCF_BUCKET}" \
    --output-recursive extracted_cache="${EXT_BUCKET}" \
    --output-recursive output="${OUTPUT_BUCKET}" \
    --command "
      set -ex
      
      # Localized paths
      PLUGIN_DIR=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/plugins
      # Use the loftee subdirectory for LoFTEE files
      HUMAN_ANCESTOR_FILE=\${PLUGIN_DIR}/human_ancestor.fa.gz
      HUMAN_ANCESTOR_INDEX=\${PLUGIN_DIR}/human_ancestor.fa.gz.fai
      CONSERVATION_FILE=\${PLUGIN_DIR}/loftee.sql
      GERP_FILE_PATH=\${PLUGIN_DIR}/gerp_conservation_scores.homo_sapiens.GRCh38.bw
      CACHE_DIR=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache
      EXTRACTED_CACHE_DIR=/mnt/data/output/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/cache/extracted
      # IMPORTANT: Remove the backslash so the variable expands.
      TEST_VCF=/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_newID/exome_v8.chr${CHR}.new_id.split_multiallelic.sites_only.pass_qc.vcf.gz
      
      # Export environment variables needed for VEP.
      export PERL5LIB=/opt/vep/src/ensembl-vep/modules:/opt/vep/src/bioperl-live:\$PERL5LIB:/mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/plugins
      export LOFTEE_DIR=\${PLUGIN_DIR}/loftee
      
      echo 'Listing contents of the plugins directory:'
      ls -lh \${PLUGIN_DIR}
      
      echo 'Listing contents of the cache directory:'
      ls -lh \${CACHE_DIR}
      
      echo 'Extracting VEP cache files:'
      CACHE_FILES=(homo_sapiens_vep_113_GRCh38.tar.gz homo_sapiens_merged_vep_113_GRCh38.tar.gz homo_sapiens_refseq_vep_113_GRCh38.tar.gz)
      for FILE in \"\${CACHE_FILES[@]}\"; do
        FULL_PATH=\"\${CACHE_DIR}/\${FILE}\"
        if [[ -f \"\${FULL_PATH}\" ]]; then
          echo \"Extracting \${FULL_PATH}...\"
          tar -xzf \"\${FULL_PATH}\" -C \"\${EXTRACTED_CACHE_DIR}\"
        else
          echo \"WARNING: File \${FULL_PATH} not found! Skipping...\"
        fi
      done
      
      echo 'Listing extracted cache contents:'
      ls -lh \${EXTRACTED_CACHE_DIR}
      
      echo 'Listing the test VCF file:'
      ls -lh \${TEST_VCF}
      
      # Define the main output file.
      OUTPUT_FILE=/mnt/data/output/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/vep/output/AOU_v8.finalAnnot.chr${CHR}.new_id.split_multiallelic.sites_only.pass_qc.txt
      
      echo 'Running VEP with the LoF plugin on chromosome ${CHR}...'
      perl /opt/vep/src/ensembl-vep/vep \\
        -i \${TEST_VCF} \\
        -o \${OUTPUT_FILE} \\
        --warning_file /tmp/vep_warnings.txt \\
        --plugin LoF,loftee_path:\${PLUGIN_DIR}/loftee,human_ancestor_fa:\${HUMAN_ANCESTOR_FILE},human_ancestor_index:\${HUMAN_ANCESTOR_INDEX},conservation_file:\${CONSERVATION_FILE},gerp_bigwig:\${GERP_FILE_PATH} \\
        --plugin CADD,snv=\${PLUGIN_DIR}/whole_genome_SNVs_inclAnno.tsv.gz,indels=\${PLUGIN_DIR}/whole_genome_SNVs_inclAnno.tsv.gz \\
        --plugin dbNSFP,\${PLUGIN_DIR}/dbNSFP/dbNSFP4.9a_grch38.gz,Ensembl_transcriptid,Uniprot_acc,VEP_canonical,LRT_pred,SIFT_pred,MutationTaster_pred,Polyphen2_HDIV_pred,Polyphen2_HVAR_pred,REVEL_score \\
        --plugin SpliceAI,snv=\${PLUGIN_DIR}/spliceai_scores.raw.snv.hg38.vcf.gz,indel=\${PLUGIN_DIR}/spliceai_scores.raw.indel.hg38.vcf.gz \\
        --everything \\
        --buffer_size 40000 \\
        --fork 24 \\
        --minimal \\
        --offline \\
        --dir_cache \${EXTRACTED_CACHE_DIR} \\
        --no_escape \\
        --fasta \${PLUGIN_DIR}/Homo_sapiens_assembly38.fasta \\
        --hgvs \\
        --cache \\
        --format vcf \\
        --force_overwrite \\
        --tab \\
        --dir_plugins \${PLUGIN_DIR}
      
      echo 'VEP run for chromosome ${CHR} completed. Check the output at:' \${OUTPUT_FILE}
    " &
}

# Launch VEP jobs for chromosomes # in parallel. This one is for 3 and 6 but adjust and add your chromosome numbers accordingly
for chr in 3 21; do
  run_vep_job $chr
done

### Convert normalized plink2 files to plink bim, fam, and bed files

In [ ]:
%%bash

# Load the aou_dsub helper
source ~/aou_dsub.bash

# Define common buckets
USER_NAME=${USER_NAME:-$(whoami)}
PLINK_BUCKET="gs://fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_plinkID"
OUTPUT_BUCKET="gs://fc-secure-ba7f8e74-e9c2-44d2-a3d0-be0aada98680/AOU_v8.exome.normalized.plink2_files/bed_bim_fam_files"
LOGGING_BUCKET="${OUTPUT_BUCKET}/logging"

# Loop over chromosomes 1–22
for CHR in {1..22}; do
  JOB_NAME="plink2-chr${CHR}-${USER_NAME}"

  echo "Submitting conversion job for chr${CHR}…"

  aou_dsub \
    --name "${JOB_NAME}" \
    --image "gcr.io/ritchie-aou-psom-9015/plink2:latest" \
    --logging "${LOGGING_BUCKET}" \
    --min-cores 4 \
    --min-ram 8 \
    --boot-disk-size 50 \
    --disk-size 1000 \
    --input INPUT_PGEN="${PLINK_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.pgen" \
    --input INPUT_PVAR="${PLINK_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.pvar" \
    --input INPUT_PSAM="${PLINK_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.psam" \
    --output OUTPUT_BED="${OUTPUT_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.bed" \
    --output OUTPUT_BIM="${OUTPUT_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.bim" \
    --output OUTPUT_FAM="${OUTPUT_BUCKET}/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc.fam" \
    --command "
      set -ex
      echo 'Converting chr${CHR} pgen → bed/bim/fam…'
      plink2 \
        --pfile /mnt/data/input/gs/fc-secure-5e490ca2-d5ae-40a3-aa5e-7355e31ab9cc/chr_plinkID/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc \
        --make-bed \
        --out /mnt/data/output/gs/fc-secure-ba7f8e74-e9c2-44d2-a3d0-be0aada98680/AOU_v8.exome.normalized.plink2_files/bed_bim_fam_files/exome_v8.chr${CHR}.new_id.split_multiallelic.pass_qc"        

done